https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb

In [1]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [2]:
header = ["Color", "Diameter", "Fruit"]

In [3]:
def class_counts(data):
    # used while computing gini coefficient
    classes = [x[-1] for x in data]
    counts = dict()
    for cls in classes:
        if cls not in counts:
            counts[cls] = 0
        counts[cls] += 1
    return counts

In [4]:
classes = class_counts(training_data)
print(classes)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [5]:
def isnumeric(key):
    # used in question to distinguish between a numercial or categorical variable
    return isinstance(key, int) or isinstance(key, float)

In [6]:
print(isnumeric(1))
print(isnumeric(7.0))
print(isnumeric('Apple'))

True
True
False


$$Gini = 1 - \Sigma_{i} p_i^2$$

Where $p_i: $ probability of $i^{th}$ class

In [7]:
def gini(data):
    counts = class_counts(data)
    impurity = 1.0
    total = sum([counts[x] for x in counts])
    for count in counts:
        p = counts[count]/total
        impurity -= p**2
    return impurity

In [8]:
print(gini(training_data))
print(gini([["Apple"],["Apple"],["Apple"],]))
print(gini([["Apple"],["Orange"],["Strawberry"],]))

0.6399999999999999
0.0
0.6666666666666665


In [9]:
# def unique_vals(training_data, idx):
#     return set([x[idx] for x in training_data])

In [10]:
# print(unique_vals(training_data, 0))
# print(unique_vals(training_data, 1))
# print(unique_vals(training_data, 2))

In [27]:
class Question:
    # Question to partition data
    def __init__(self, header, value):
        self.header = header
        self.value = value
    
    def match(self, example):
        if isnumeric(self.value):
            return example[self.header] >= self.value
        else:
            return example[self.header]== self.value
    
    def __repr__(self):
        if isnumeric(self.value):
            condition = ">="
        else:
            condition = "=="
        return f"Is {header[self.header]} {condition} {self.value}?"

In [12]:
q = Question(0,"Green")
print(q)
print(training_data[0])
print(q.match(training_data[0]))
print(training_data[1])
print(q.match(training_data[1]))

Is Color == Green?
['Green', 3, 'Apple']
True
['Yellow', 3, 'Apple']
False


In [28]:
def partition(data, question):
    # partitin into true and false branches based on the question
    left = []
    right = []
    for sample in data:
        if question.match(sample):
            left.append(sample)
        else:
            right.append(sample)
    return left, right

In [14]:
left, right = partition(training_data, q)
print(q)
print(left)
print(right)

Is Color == Green?
[['Green', 3, 'Apple']]
[['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


In [29]:
def info_gain(current_impurity, left, right):
    # Information gain obtained by current split
    p_left = len(left)/ (len(left)+len(right))
    p_right = 1-p_left
    gini_left = gini(left)
    gini_right = gini(right)
    return current_impurity - p_left*gini_left - p_right*gini_right

In [30]:
current_impurity = gini(training_data)
left, right = partition(training_data, Question(0, "Green"))
print(info_gain(current_impurity, left, right))
left, right = partition(training_data, Question(1, 3))
print(info_gain(current_impurity, left, right))

0.1399999999999999
0.37333333333333324


In [31]:
def find_best_split(rows, debug = False):
    # find best split for the given data
    n_cols = len(rows[0])-1
    best_gain = 0
    best_question = None
    current_impurity = gini(rows)
    for col in range(n_cols):
        unique_cols = set([x[col] for x in rows])
        if debug:
            print(unique_cols)
        for val in unique_cols:
            question = Question(col, val)
            left, right = partition(rows, question)
            if len(left) == 0 or len(right) == 0: #skip the question if it doesn't partition the data
                continue
            current_gain = info_gain(current_impurity, left, right)
            if debug:
                print(current_gain, question)
            if current_gain >= best_gain:
                best_gain = current_gain
                best_question = question
    return best_gain, best_question
            

In [32]:
find_best_split(training_data, debug=True)

{'Green', 'Yellow', 'Red'}
0.1399999999999999 Is Color == Green?
0.17333333333333323 Is Color == Yellow?
0.37333333333333324 Is Color == Red?
{1, 3}
0.37333333333333324 Is Diameter >= 3?


(0.37333333333333324, Is Diameter >= 3?)

In [19]:
class LeafNode:
    def __init__(self, rows):
        # get counts of classes
        counts = class_counts(rows)
        # convert counts to class probabilities
        total = sum(list(counts.values()))
        probs = {}
        for key in counts:
            probs[key] = counts[key]/total
        self.predictions = probs

In [20]:
class DecisionNode:
    def __init__(self, question, true_partition, false_partition):
        self.question = question
        self.true_partition = true_partition
        self.false_partition = false_partition

In [34]:
def build_tree(rows):
    # recursively build tree
    best_gain, question = find_best_split(rows)
    # terminating condition
    if best_gain == 0:
        return LeafNode(rows)
    # recursively partition rows
    true_branch, false_branch = partition(rows, question)
    # build left branch
    left_branch = build_tree(true_branch)
    # build right branch
    right_branch = build_tree(false_branch)
    return DecisionNode(question, left_branch, right_branch)

In [22]:
def print_tree(node, spacing=""):
    if isinstance(node, LeafNode):
        print(f"{spacing} {node.predictions}")
        return
    print(f"{spacing} {node.question}")
    # left tree
    print(f"{spacing} --> True")
    print_tree(node.true_partition, spacing+"\t")
    # right tree
    print(f"{spacing} --> False")
    print_tree(node.false_partition, spacing+"\t")   

In [23]:
tree = build_tree(training_data)
print_tree(tree)

 Is Diameter >= 3?
 --> True
	 Is Color == Yellow?
	 --> True
		 {'Apple': 0.5, 'Lemon': 0.5}
	 --> False
		 {'Apple': 1.0}
 --> False
	 {'Grape': 1.0}


In [24]:
def classify(node, row):
    if isinstance(node, LeafNode):
        return node.predictions
    else:
        if node.question.match(row):
            return classify(node.true_partition, row)
        else:
            return classify(node.false_partition, row)

In [25]:
for row in training_data:
    print(f"{row}: Prediction: {classify(tree, row)}")

['Green', 3, 'Apple']: Prediction: {'Apple': 1.0}
['Yellow', 3, 'Apple']: Prediction: {'Apple': 0.5, 'Lemon': 0.5}
['Red', 1, 'Grape']: Prediction: {'Grape': 1.0}
['Red', 1, 'Grape']: Prediction: {'Grape': 1.0}
['Yellow', 3, 'Lemon']: Prediction: {'Apple': 0.5, 'Lemon': 0.5}


In [26]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
for row in testing_data:
    print(f"{row}: Prediction: {classify(tree, row)}")

['Green', 3, 'Apple']: Prediction: {'Apple': 1.0}
['Yellow', 4, 'Apple']: Prediction: {'Apple': 0.5, 'Lemon': 0.5}
['Red', 2, 'Grape']: Prediction: {'Grape': 1.0}
['Red', 1, 'Grape']: Prediction: {'Grape': 1.0}
['Yellow', 3, 'Lemon']: Prediction: {'Apple': 0.5, 'Lemon': 0.5}
